This is a program which is in testing phase. We will use a linear regression ML model to give an SOC number for the flights.

In [28]:
# Import the querying module
from flight_querying import query_flights
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Set up and retrieve the data from the database.
db_connect = query_flights()

In [29]:
# Make the data dictionary holder
data_dict = {
    "Time Delta": [],
    "SOC Delta": [],
    "Activity": [],
    "Average Power": [],
    "Id": [],
    "Unique Data Identifier": [],
    "temperature": [],
    "Visibility": [],
    "Wind Speed": []
}

# Create a increasing int variable to keep track of the Unique Data Identifier
unique_identifier = 0

In [30]:
def data_parser(parsing_dataframe):

    global unique_identifier

    # Get the current exercise
    current_exercise = parsing_dataframe.iloc[0, parsing_dataframe.columns.get_loc('activity')]
    id = parsing_dataframe.iloc[0, parsing_dataframe.columns.get_loc('id')]
    max_soc, min_soc = 0, 101
    power_list = [0]
    min_time, max_time = 100, 0

    # iterate over all the rows
    for index, row in parsing_dataframe.iterrows():

        # Get the data needed from the rows. Append the power
        new_exercise = row["activity"]
        soc = row["soc"]
        power_list.append(row["power"])
        time = row["time"]
        outside_temp = row["temperature"]
        visibility = row["visibility"]
        wind_speed = row["wind_speed"]

        # If the exercise changes or if the rows end.
        if current_exercise != new_exercise or index == len(parsing_dataframe) - 1:

            # Set the values 
            data_dict["Time Delta"].append(round(max_time - min_time, 2))
            data_dict["SOC Delta"].append(max_soc - min_soc)
            data_dict["Activity"].append(current_exercise)
            data_dict["Average Power"].append(round(sum(power_list)/len(power_list), 2))
            data_dict["temperature"].append(outside_temp)
            data_dict["Id"].append(id)
            data_dict["Visibility"].append(visibility)
            data_dict["Wind Speed"].append(wind_speed)
            data_dict["Unique Data Identifier"].append(unique_identifier)
            unique_identifier = unique_identifier + 1

            # Reset all the values
            max_soc, min_soc = soc, soc
            power_list.clear()
            max_time, min_time = time, time
        
        # SOC
        if soc >= max_soc:
            max_soc = soc
        if soc <= min_soc:
            min_soc = soc

        # TIME
        if time >= max_time:
            max_time = time
        if time <= min_time:
            min_time = time
        
        # Change current exercise
        current_exercise = new_exercise

In [31]:
# Flight IDs to include
flight_ids = [4620, 4929, 4940, 5019, 5021, 5034]

# Fetch data for specified flight IDs
# Remove NA values from each dataframe in the list and put it through the data parser
for ids in flight_ids:
    frame = db_connect.connect_flight_for_ml_data_prescription(ids)
    frame = frame[frame["activity"] != "NA"].reset_index()
    data_parser(frame)

# Concatenate data frames and shuffle the data
# all_data = pd.concat(data_frames, axis=0).sample(frac=1, random_state=42)
all_data = pd.DataFrame(data_dict)

In [32]:
# Split the data into train and test sets
train_data, test_data = train_test_split(all_data, test_size=0.3, random_state=42)

In [33]:
all_data.head()

,Time Delta,SOC Delta,Activity,Average Power,Id,Unique Data Identifier,temperature,Visibility,Wind Speed
0,1.0,3,takeoff,53.39,4620,0,66.2,9.0,4
1,0.9,4,climb,51.92,4620,1,66.2,9.0,4
2,1.3,2,cruise,15.35,4620,2,64.4,9.0,6
3,1.2,0,landing,0.08,4620,3,64.4,9.0,6
4,0.7,3,takeoff,53.31,4620,4,66.2,9.0,4


In [34]:
print(f"Length train_x = {len(train_data)} \n Length train_y = {len(test_data)}")

Length train_x = 67 
 Length train_y = 29


One-Hot-Encoding of the Operations columns

In [35]:
# ONE-HOT ENCODE
# https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/
def one_hot(df, col, pre):
  encoded = pd.get_dummies(df[col], prefix=pre)
  for column in encoded:
    encoded = encoded.rename(columns={column: col + "_" + column})
  encoded['Unique Data Identifier'] = df['Unique Data Identifier']
  return encoded

In [36]:
# Encode Train data
train_encoded = one_hot(train_data, "Activity", 'is')
final_train_x = pd.merge(train_data, train_encoded, on=["Unique Data Identifier"])
final_train_y = final_train_x["SOC Delta"].to_numpy()
final_train_x = final_train_x.drop(columns=["SOC Delta", "Id", "Unique Data Identifier", "Activity"])

# Encode Test data
test_encoded = one_hot(test_data, "Activity", 'is')
final_test_x = pd.merge(test_data, test_encoded, on=["Unique Data Identifier"])
final_test_y = final_test_x["SOC Delta"].to_numpy()
final_test_x = final_test_x.drop(columns=["SOC Delta", "Id", "Unique Data Identifier", "Activity"])

In [37]:
final_train_x

,Time Delta,Average Power,temperature,Visibility,Wind Speed,Activity_is_climb,Activity_is_cruise,Activity_is_descent,Activity_is_landing,Activity_is_power off stall,Activity_is_power on stall,Activity_is_slow flight,Activity_is_steep turns,Activity_is_takeoff
0,0.7,29.97,44.6,9.0,3,False,False,False,False,False,False,False,True,False
1,2.3,43.48,53.6,7.0,4,True,False,False,False,False,False,False,False,False
2,1.1,44.94,53.6,9.0,7,True,False,False,False,False,False,False,False,False
3,0.8,0.02,53.6,9.0,7,False,False,True,False,False,False,False,False,False
4,1.0,44.63,64.4,9.0,6,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,0.7,44.04,44.6,9.0,2,True,False,False,False,False,False,False,False,False
63,3.4,17.77,48.2,9.0,3,False,False,False,False,False,False,True,False,False
64,1.5,20.41,66.2,9.0,4,False,True,False,False,False,False,False,False,False
65,0.7,53.09,57.2,9.0,10,False,False,False,False,False,False,False,False,True


In [38]:
print(f"Length test_encoded = {len(test_encoded)} \n Length train_encoded = {len(train_encoded)}")

Length test_encoded = 29 
 Length train_encoded = 67


In [39]:
print(f"Length train_x = {len(final_train_x)} \n Length train_y = {len(final_train_y)}")

Length train_x = 67 
 Length train_y = 67


In [40]:
print(f"Length test_x = {len(final_test_x)} \n Length test_y = {len(final_test_y)}")

Length test_x = 29 
 Length test_y = 29


Machine Learning Model Implementation

In [76]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Standardize the features (optional but recommended for neural networks)
scaler = StandardScaler()
final_train_x = scaler.fit_transform(final_train_x)
final_test_x = scaler.transform(final_test_x)

# Convert NumPy arrays to PyTorch tensors
train_x_tensor = torch.FloatTensor(final_train_x)
train_y_tensor = torch.FloatTensor(final_train_y)
test_x_tensor = torch.FloatTensor(final_test_x)
test_y_tensor = torch.FloatTensor(final_test_y)

# Convert labels to float tensor
train_y_tensor = train_y_tensor.float()
test_y_tensor = test_y_tensor.float()

# Create a custom neural network class
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(final_train_x.shape[1], 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Instantiate the model
model = SimpleNN()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005) #Adam (short for Adaptive Moment Estimation) is an optimization algorithm commonly used for training deep neural networks. 
                                                     #It is an extension of the stochastic gradient descent (SGD) optimization algorithm with adaptive learning rates for each parameter.

# Convert data to DataLoader for batching
train_data = TensorDataset(train_x_tensor, train_y_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Training loop
epochs = 30
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()



/Users/nayeema/envs/venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/nayeema/envs/venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [77]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import r2_score

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(test_x_tensor)
    mse = mean_squared_error(final_test_y, y_pred)
    r2 = r2_score(final_test_y, y_pred)
    print(f"Mean Squared Error: {mse:.5f}")
    print(f"R^2 Score: {r2:.5f}")

Mean Squared Error: 12.41190
R^2 Score: 0.06264


In [42]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import r2_score

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(test_x_tensor)
    mse = mean_squared_error(final_test_y, y_pred.numpy())
    r2 = r2_score(final_test_y, y_pred.numpy())
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R^2 Score: {r2:.2f}")

Mean Squared Error: 22.54
R^2 Score: -0.70


In [49]:
# Evaluation
model.eval()

# Get the weights from the model
weights_fc1 = model.fc1.weight.detach().numpy().flatten()
weights_fc2 = model.fc2.weight.detach().numpy().flatten()
weights_fc3 = model.fc3.weight.detach().numpy().flatten()

# Combine weights and feature names
weights = weights_fc1.tolist() + weights_fc2.tolist() + weights_fc3.tolist()
feature_names = ["Time Delta", "Average Power", "Activity_is_climb", "Activity_is_cruise", 
                 "Activity_is_descent", "Activity_is_landing", "Activity_is_power on stall", 
                 "Activity_is_power off stall", "Activity_is_slow flight", "Activity_is_steep turns", 
                 "Activity_is_takeoff"]

# Print the features and their corresponding weights
print("Feature\tWeight")
for feature, weight in zip(feature_names, weights):
    print(f"{feature}\t{weight:.6f}")

Feature	Weight
Time Delta	0.032977
Average Power	0.215463
Activity_is_climb	-0.064137
Activity_is_cruise	0.170721
Activity_is_descent	0.084095
Activity_is_landing	-0.229632
Activity_is_power on stall	-0.220558
Activity_is_power off stall	0.180566
Activity_is_slow flight	-0.074359
Activity_is_steep turns	-0.232809
Activity_is_takeoff	-0.042155
